# Ejercicio 3 Tweet Generation
Ya se hizo la limpieza de datos inicial de este dataset en el notebook [Analisis Dataset 2](00_2_AnalisisDataset2.ipynb)

En este notebook generaremos tweets haciendo que se parezcan a los que twitea Trump, para ello, usaremos Language Modeling con Deep Learning.

In [2]:
!python -m spacy download en_core_web_sm

import warnings
warnings.filterwarnings('ignore')

import gensim
import re
import pyLDAvis.gensim
import io
import spacy
import en_core_web_sm
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import io
import pylab as pl
from random import shuffle, choice, sample
from sklearn.model_selection import StratifiedShuffleSplit
from copy import copy
from IPython import display
from string import punctuation
from stop_words import get_stop_words

%matplotlib inline
%load_ext autoreload
%autoreload 2
%matplotlib notebook

from keras.models import Model, Sequential
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
from keras.layers import Bidirectional
from keras.layers import Embedding
from keras.layers import Dot, Concatenate, Flatten, Permute, Multiply, dot, concatenate
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.preprocessing import sequence
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.models import load_model

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
c:\Users\asens\.vscode\extensions\ms-python.python-2020.2.64397\pythonFiles\lib\python\past\types\oldstr.py:36: DeprecationWarning: invalid escape sequence \d
  """
D:\Programas\Anaconda3\envs\tf-gpu\lib\site-packages\h5py\_hl\base.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import (Mapping, MutableMapping, KeysView,
D:\Programas\Anaconda3\envs\tf-gpu\lib\site-packages\h5py\_hl\base.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import (Mapping, MutableMapping, KeysView,
D:\Programas\Anaconda3\envs\tf-gpu\lib\site-packages\h5py\_hl\base.py:19: DeprecationWarning: Using or importing the ABCs from 'collecti

In [3]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.gpu_options.per_process_gpu_memory_fraction = 0.95
config.gpu_options.visible_device_list = "0"

# Comando nvidia-smi para ver la lista de GPU

set_session(tf.Session(config=config))  # set this TensorFlow session as the default session for Keras

In [4]:
# Permitir ver columnas completas
pd.set_option('display.max_colwidth', None)

# Función para cargar dataframe
def load_data(ruta, nombre):
    path = os.path.join(ruta, nombre)    
    return pd.read_csv(path, encoding="ISO-8859-1")

In [5]:
# Carga de datos
df = load_data('data', 'dataset_2_limpieza_inicial.csv')
df.head()

,president,tweet
0,OBAMA,low plastic stool cheap delicious noodles cold hanoi beer ill remember tony teach us food impoantly ability bring us together make us little less afraid unknown well miss
1,OBAMA,low plastic stool cheap delicious noodles cold hanoi beer ill remember tony teach us food impoantly ability bring us together make us little less afraid unknown well miss
2,OBAMA,national gun violence awareness day show commitment keep kid safe gun violence everyone weve lose take action change leadership laws reflect commitment matter long take
3,OBAMA,never truly debt owe fall heroes remember honor sacrifice live endure ideals justice equality oppounity generations americans give last full measure devotion
4,OBAMA,center leaders tomorrow ready step build world michelle grateful chicago city council make happen


## Seleccionamos los tweets de Trump

In [6]:
df = df[df['president'] == 'TRUMP']
df

,president,tweet
2892,TRUMP,thoughts prayers families serviceman kill fellow servicemen wound somalia truly heroes
2893,TRUMP,thoughts prayers families serviceman kill fellow servicemen wound somalia truly heroes
2894,TRUMP,head canada grinÃ talk mostly center long time unfair trade unite state go singapore talk noh korea denuclearization wont talk russian witch hunt hoax
2895,TRUMP,congratulations washington capitals great play win cup championship alex team spectacular true dc many ways time
2896,TRUMP,look forward unfair trade deal grinÃ countries doesnt happen come even better
...,...,...
5874,TRUMP,vast money nato amp unite state must pay powerful expensive defense provide germany
5875,TRUMP,despite hear fake news great meet chancellor angela merkel germany
5876,TRUMP,great meet committee morning
5877,TRUMP,president change small confidence


In [7]:
trump_dataset = list()

for index, row in df.iterrows():
    sentence = row[1]
    trump_dataset.append(sentence)

len(trump_dataset), trump_dataset[0]

(2987,
 'thoughts prayers families serviceman kill fellow servicemen wound somalia truly heroes')

## Tokenizar a nivel de carácter en lugar de palaba

In [8]:
tokenized = [list(x) for x in trump_dataset]

In [9]:
init_chars = [x[:5] for x in tokenized] # me quedo con los 4 caracteres iniciales

for i in range(len(init_chars)):
    tmp = init_chars[i]
    tmp.insert(0, '<SOS>')
    init_chars[i] = tmp[:5]
    
init_chars[0]

['<SOS>', 't', 'h', 'o', 'u']

Obtengo la longitud máxima de las frases y la longitud media

In [10]:
maxlen = max([len(x) for x in tokenized])
avglen = sum([len(x) for x in tokenized])/len(tokenized)
print('Longitud maxima: ', maxlen)
print('Media de Longitud: ', avglen)

Longitud maxima:  219
Media de Longitud:  89.00836960160696


Genero el vocabulario

In [13]:
total_tokens = [t for s in trump_dataset for t in s]
print('Numero de tokens: {}'.format(len(total_tokens)))

Numero de tokens: 265868


In [14]:
from collections import Counter
vocab_counter = Counter(total_tokens)
vocab = [w for w, v in vocab_counter.items() if v > 2]  # Caracteres que mínimo aparezcan 2 veces
vocab = ['<PAD>', '<UNK>', '<SOS>', '<EOS>'] + vocab

nb_vocab = len(vocab)
nb_vocab

34

In [16]:
c2id = {k:i for i, k in enumerate(vocab)}
id2w = {i:k for k, i in c2id.items()}

In [17]:
maxlen = 5

step = 1

data_train = []

for x in tokenized:
    x.insert(0, '<SOS>')
    x.append('<EOS>')
    for i in range(0, len(x)-maxlen, step):
        data_train.append((x[i:i+maxlen], x[i+maxlen]))
        
len(data_train)

256907

In [27]:
SAMPLE_EVERY = 4

## Helper functions

In [28]:
def sample_pred(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [29]:
class Sampletest(Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % SAMPLE_EVERY == 0  and epoch>0:
            data_test = []
            nb_samples = 1
            
            params = {
                'maxlen': maxlen,
                'vocab': nb_vocab,
                'use_embeddings': True
                }
            for _ in range(nb_samples):
                data_test = choice(init_chars)
                for diversity in [0.2, 0.6, 1.2]:
                    print('----- diversity:', diversity)
                    sentence = copy(data_test)
                    generated = copy(data_test)
                    for i in range(len(data_test), 400):
                        x_pred = np.zeros((1, params['maxlen']))
                        for t, char in enumerate(sentence):
                            x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                        preds = self.model.predict(x_pred, verbose=0)[0]
                        next_index = sample_pred(preds, diversity)
                        next_char = id2w[next_index]
                        if next_char == '<EOS>':
                            break
                        generated += [next_char]
                        sentence = sentence[1:] 
                        sentence += [next_char]
                    print(''.join(generated))

In [30]:
class HistoryDisplay(Callback):
    
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []
        self.epochs = []
        self.fig, self.ax = plt.subplots()
        #plt.show()
        
        plt.ion()
        self.fig.show()
        self.fig.canvas.draw()
    
    def on_epoch_end(self, epoch, logs):
        self.epochs.append(epoch)
        self.losses.append(logs['loss'])
        self.accs.append(logs['acc'])
        if epoch % PLOT_EVERY == 0:
            
            self.ax.clear()
            self.ax.plot(self.epochs, self.accs, 'g', label='acc')
            self.ax.plot(self.epochs, self.losses, 'b', label='loss')
            legend = self.ax.legend(loc='upper right', shadow=True, fontsize='x-large')
            #display.clear_output(wait=True)
            #display.display(pl.gcf())
            self.fig.canvas.draw()
            
            #plt.draw()

In [31]:
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

## Decidir arquitectura y preparar el train y el predict

In [32]:
class LM:
    def __init__(self, **kwargs):
        self.params = kwargs.pop('params', None)
    
    def compile_bidirectional(self, params={}):
        lm_inputs = Input(shape=(params['maxlen'], ), name='lm_input')
        embeddings = Embedding(params['vocab'], params['emb_feats'])(lm_inputs)
        lstm =  CuDNNLSTM(params['rnn_hidden'], return_sequences=True, name='rnn1')        
        
        lmlstm = Bidirectional(lstm)(embeddings)       
        
        stacklstm =  CuDNNLSTM(params['rnn_hidden'], return_sequences=False, name='stacked')
        stackedlstm = stacklstm(lmlstm)
        
        lmout = Dense(params['vocab'], activation='softmax')(stackedlstm)
        
        model = Model(lm_inputs, lmout)
        
        model.compile(
            optimizer='rmsprop', 
            loss='categorical_crossentropy', 
            metrics=['accuracy']
        )
        
        model.summary()
        
        return model
        
    def train(self, model, data, params={}):
        callbacks = self._get_callbacks()
        
        if 'shuffle' in params and params['shuffle']:
            shuffle(data)
            
        sentences, next_chars = zip(*data)
        #print(sentences[0])

        x = np.zeros((len(data), params['maxlen']))
        y = np.zeros((len(data), params['vocab']))

        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                x[i, t] = c2id[char] if char in c2id else c2id['<UNK>']
            y[i, c2id[next_chars[i]] if next_chars[i] in c2id else c2id['<UNK>']]  = 1
        
        model.fit(x, y, batch_size=params['batch_size'], epochs=params['epochs'], callbacks=callbacks, verbose=1)

    def predict(self, model, data, params={}):        
        if 'use_embeddings' in params and params['use_embeddings']:
            # variedad en las predicciones
            for diversity in [0.2, 0.6, 1.2]:
                print('----- diversity:', diversity)
                sentence = copy(data)
                generated = copy(data)
                # cuantas predicciones queremos hacer
                for i in range(len(data), 400):
                    x_pred = np.zeros((1, params['maxlen']))
                    # preparar inpunt
                    for t, char in enumerate(sentence):
                        x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
                    # predecir
                    preds = self.model.predict(x_pred, verbose=0)[0]
                    next_index = sample_pred(preds, diversity)
                    next_char = id2w[next_index]
                    # mirar si hemos terminado
                    if next_char == '<EOS>':
                        break
                    # ana                        
                    generated += [next_char]
                    sentence = sentence[1:] 
                    sentence += [next_char]
                print(''.join(generated))
    
    
    def load(self, model_path='seq2seq_attn.h5'):
        return load_model(model_path)
    
    def _get_callbacks(self, model_path='seq2seq_attn.h5'):
        es = EarlyStopping(monitor='loss', patience=4, mode='auto', verbose=0)       
        
        save_best = ModelCheckpoint(model_path, monitor='loss', verbose = 0, save_best_only=True, save_weights_only=False, period=2)
        st = Sampletest()
        # hd = HistoryDisplay()
        rlr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)
        return [st, rlr]

## Hyperparámetros

In [33]:
compile_params = {
    'maxlen': maxlen, 
    'vocab': len(vocab),
    'emb_feats': 100,
    'rnn_hidden': 256,
}

## Compilar

In [34]:
lm = LM()
lm_model = lm.compile_bidirectional(params=compile_params)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lm_input (InputLayer)        (None, 5)                 0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 5, 100)            3400      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 512)            733184    
_________________________________________________________________
stacked (CuDNNLSTM)          (None, 256)               788480    
_________________________________________________________________
dense_2 (Dense)              (None, 34)                8738      
Total params: 1,533,802
Trainable params: 1,533,802
Non-trainable params: 0
_________________________________________________________________


## Entrenar

In [35]:
train_params = {
    'epochs': 300,
    'batch_size': 512,
    'shuffle': True,
    'vocab': nb_vocab,
    'maxlen': maxlen,
    'use_embeddings': True
}

lm.train(model=lm_model, data=data_train, params=train_params)

 hard meet change make american people amp stock market hit another make america great honor welcome president trump campaign proud state terrorist attack london strong well
----- diversity: 0.6
<SOS>history
----- diversity: 1.2
<SOS>history amp begin repeal map thank give news away service charge clinton morning
Epoch 210/300
256907/256907 [==============================] - 12s 46us/step - loss: 0.7061 - acc: 0.7290
Epoch 211/300
256907/256907 [==============================] - 12s 46us/step - loss: 0.7061 - acc: 0.7288
Epoch 212/300
256907/256907 [==============================] - 12s 46us/step - loss: 0.7061 - acc: 0.7292
Epoch 213/300
256907/256907 [==============================] - 12s 46us/step - loss: 0.7060 - acc: 0.7291
----- diversity: 0.2
<SOS>fake news media work hard believe fake news media cover amp deductibles way get read schedule time high would shape devin mccabe clinton campaign problem lie show collusion one years talk action people want take place today offer condo

## Resumen
Por algún motivo se ha borrado el log anterior a la época 210, no tengo tiempo de volver a generarlo.

Si miramos los datos que genera el entrenamiento, comprobamos que los tweets generados con diversidad mayor a 0.2 dejan de tener sentido.

## Testear el modelo
Probamos con diversidad 0.2 a generar 5 tweets

In [37]:
nb_samples = 5

params = {
    'maxlen': maxlen,
    'vocab': nb_vocab,
    'use_embeddings': True
    }

for _ in range(nb_samples):
    data_test = choice(init_chars)

    # Cuanto mas se acerque diversity a 1, menos sentido tendran los tweets
    for diversity in [0.2]:
        print('----- diversity:', diversity)
        sentence = copy(data_test)
        generated = copy(data_test)
        for i in range(len(data_test), 400):
            x_pred = np.zeros((1, params['maxlen']))
            for t, char in enumerate(sentence):
                x_pred[0, t] = c2id[char] if char in c2id else c2id['<UNK>']
            preds = lm_model.predict(x_pred, verbose=0)[0]
            next_index = sample_pred(preds, diversity)
            next_char = id2w[next_index]
            if next_char == '<EOS>':
                break
            generated += [next_char]
            sentence = sentence[1:] 
            sentence += [next_char]
        tweet = ''.join(generated)
        print(tweet[5:])

----- diversity: 0.2
poor economy boom get today offer condolences terrorist attack london strong well
----- diversity: 0.2
general meet president trump campaign proud state congress make america great honor welcome amp replace obama administer theresa may today offer condolences terrorist attack london strong well
----- diversity: 0.2
spect country must always president money spend military state america great honor sign trillion dollars see saudi arabia peace memos clinton investigation best tax cut american people great meet president xi china already protect country state trump campaign problem didnt want state terrorist attack london strong well
----- diversity: 0.2
experimentally great honor welcome president obama administer theresa may today offer condolences terrorist attack london strong well
----- diversity: 0.2
determinate many years talk action congress make american people thing russia coverage amp deduction amp replace obama administer theresa may today offer condolences